In [19]:
import pandas as pd
import nltk
# nltk.download('all')
import re
import numpy as np
import collections
import sklearn as sk
import string

In [20]:
def data_cleaning(raw_data):
    '''
    This function takes in a string specifically "cyberpunk" and delete it.
    '''
    useful_words = raw_data.lower().split()
    useful_words = [w.replace('cyberpunk', '') for w in useful_words]
    return( " ".join(useful_words))

In [21]:
df = pd.read_csv("../../B_Data_Cleaning/synthetic_reviews_all_trial_1.csv")
df["Review"] = df["Review"].astype("str")
df['Review'] = df['Review'].apply(data_cleaning)
df["Review"] = df["Review"].astype("str")

In [22]:
# make a set_column to count the number of words in each review
df["set_column"] = df["Review"].apply(lambda x: set(x.split()))
df

,Review,Recommended or Not Recommended,set_column
0,idols hex scifistoryelements airborne underwor...,True,"{sandwish, storry, neondiarrhea, artistically,..."
1,ablazing ｉｔｓ alanah дрянь ammunition unbreakab...,True,"{mau, highrises, göre, узнать, adjusting, cohe..."
2,doubletapping overcrowded critiquer minesweepe...,True,"{transição, orçamento, ruleset, crates, entail..."
3,puffing mild miniboss vybouchnou selected bonl...,True,"{maniacs, kaoru, 월등하게, citizenscops, wham, rai..."
4,rapid bpoliceb seeen contagion whazoo,True,"{rapid, whazoo, seeen, bpoliceb, contagion}"
...,...,...,...
16570,gathering acording damagedealing hquests guide...,False,"{humanityethics, buggyness, exploitation, ashe..."
16571,ghameplay houdini viks chews planed ao corpotr...,False,"{touch, exploitation, unrealized, karaoke, unn..."
16572,oldgen excelentes httpswwwyoutubecomwatchvzfrc...,False,"{battlegun, aynı, customizável, lasts, somethi..."
16573,preferência fixespolish compli platoon caminan...,False,"{unaccetable, entail, strat, countersteering, ..."


In [23]:
# keep only 5250 reviews for each based on 'Recommended or Not Recommended" column
temp_recom = df.loc[df["Recommended or Not Recommended"] == True,:]
temp_recom = temp_recom.sample(5250).reset_index(drop=True)

temp_not_recom = df.loc[df["Recommended or Not Recommended"] == False,:]
temp_not_recom = temp_not_recom.sample(5250).reset_index(drop=True)

# combine temp_recom and temp_not_recom
temp = pd.concat([temp_recom, temp_not_recom], axis=0).reset_index(drop=True)

# split the data into train and test
train, test = sk.model_selection.train_test_split(temp, test_size=0.33, random_state=42)

# # split df_recom into train and test
# df_recom = temp_recom.sample(frac=0.67, random_state=42).reset_index(drop=True)
# test_recom = temp_recom.drop(df_recom.index).reset_index(drop=True)
# print(f"train size df_recom: {len(df_recom)}, test size df_recom: {len(test_recom)}")

# # split df_not_recom into train and test
# df_not_recom = temp_not_recom.sample(frac=0.33, random_state=42).reset_index(drop=True)
# test_not_recom = temp_not_recom.drop(df_not_recom.index).reset_index(drop=True)
# print(f"train size df_not_recom: {len(df_not_recom)}, test size df_not_recom: {len(test_not_recom)}")

# # combine test_recom and test_not_recom
# test = pd.concat([test_recom, test_not_recom], axis=0).reset_index(drop=True)

In [24]:
# Dividing dataset into recommended and not recommended
df_recom = train.loc[train["Recommended or Not Recommended"] == True,:]
df_recom = df_recom.reset_index(drop=True)

df_not_recom = train.loc[train["Recommended or Not Recommended"] == False,:]
df_not_recom = df_not_recom.reset_index(drop=True)

In [25]:
# This is to make the set dictionary
# count how many times each word appears in the reviews
bow_recom_temp = collections.Counter([y for x in df_recom.set_column for y in x])
bow_not_recom_temp = collections.Counter([y for x in df_not_recom.set_column for y in x])

bow_recom_set = dict(bow_recom_temp)
bow_not_recom_set = dict(bow_not_recom_temp)

for key in bow_recom_set:
    if key not in bow_not_recom_set:
        bow_not_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_set:
    if key not in bow_recom_set:
        bow_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_recom_set:
    bow_recom_set[key] += 1

for key in bow_not_recom_set:
    bow_not_recom_set[key] += 1


In [26]:
# This is to make the list dictionary
# count how many times each word appears in the reviews
bow_recom = collections.Counter([y for x in df_recom.Review for y in x.split()])
bow_not_recom = collections.Counter([y for x in df_not_recom.Review for y in x.split()])

bow_recom_dict = dict(bow_recom)
bow_not_recom_dict = dict(bow_not_recom)

for key in bow_recom_dict:
    if key not in bow_not_recom_dict:
        bow_not_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_dict:
    if key not in bow_recom_dict:
        bow_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_recom_dict:
    bow_recom_dict[key] += 1

for key in bow_not_recom_dict:
    bow_not_recom_dict[key] += 1
    

In [27]:
bow_recom_sum_vals = sum(bow_recom_dict.values())
bow_not_recom_sum_vals = sum(bow_not_recom_dict.values())

for i in bow_recom_dict:

    bow_recom_dict[i] /= bow_recom_sum_vals

for i in bow_not_recom_dict:

    bow_not_recom_dict[i] /= bow_not_recom_sum_vals

In [28]:
assert len(bow_recom_dict) == len(bow_not_recom_dict)
assert len(bow_recom_set) == len(bow_not_recom_set)

In [29]:
test.drop(columns=["set_column"], inplace=True)

In [30]:
def define_sentiment(element, test_all=False, test_only_tf_abs=False, test_only_idf_abs=False, test_only_prob=False, test_all_abs=False):

    """This function takes a review and returns the label for that review"""

    full_shape = df.shape[0]
    positive_review_probabiliy = len(df_recom)/full_shape
    negative_review_probabiliy = len(df_not_recom)/full_shape
    
    df_choices_positive = [positive_review_probabiliy, bow_recom_set, bow_recom_dict, df_recom]
    df_choices_negative = [negative_review_probabiliy ,bow_not_recom_set, bow_not_recom_dict, df_not_recom]

    def classifier(element, your_class = 'positive'):
        flag = {}
        flag_count = 0
        if your_class == 'positive':
            df_choices = df_choices_positive
        else:
            df_choices = df_choices_negative
        prob_of_class = df_choices[0]/full_shape
        score = 1 * prob_of_class
        score = float(format(score, '.12f'))
        # score = 0.1
        for i in element.split():
            if i not in df_choices[2].keys():
                pass
            else:
                prob_word_given_class = (df_choices[2])[i]
                prob_word_given_class = float(format(prob_word_given_class, '.12f'))
                # Almost the same value, given our spin on this application. 
                # Normally, this term frequency would be calculated differently across the positive and negative documents
                # but we are only looking at the reviews as the documents themselves to determine a word's relevance in the positive
                # or negative corpus. 
                tf = np.log(prob_word_given_class)
                # tf = float(format(tf, '.12f'))
                # tf = abs(np.log(prob_word_given_class))
                # The IDF is the number of reviews / the number of reviews that contain the word in that given corpus
                # idf = abs(np.log(df_choices[3].shape[0]/(df_choices[1])[i]))
                idf = np.log(df_choices[3].shape[0]/(df_choices[1])[i])
                # idf = float(format(idf, '.12f'))
                if test_all:
                    score *= prob_word_given_class*tf*idf
                elif test_only_tf_abs:
                    score *= prob_word_given_class*tf
                    score = abs(score)
                elif test_only_idf_abs:
                    score *= prob_word_given_class*idf
                    score = abs(score)
                elif test_only_prob:
                    score *= prob_word_given_class
                elif test_all_abs:
                    score *= prob_word_given_class*tf*idf
                    score = abs(score)

                if score < 0:
                    flag[flag_count] = (score, your_class)
                    # print(flag_count,i)
                    flag_count += 1
                    # return 'Flag'


        return score
    positive_score = classifier(element, 'positive')
    negative_score = classifier(element, 'negative')
    if positive_score > negative_score:
        return True
    elif positive_score == negative_score:
        # Choosing an arbitrary value, because we assume that a review with one or few words of little substance
        # is implied to be negative, as is usual with netizens. 
        return False
    else:
        return False

In [31]:
test["score_all"] = test.Review.apply(define_sentiment, test_all=True)
test["score_tf"] = test.Review.apply(define_sentiment, test_only_tf_abs=True)
test["score_idf"] = test.Review.apply(define_sentiment, test_only_idf_abs=True)
test["score_freq"] = test.Review.apply(define_sentiment, test_only_prob=True)
test["score_mod_all"] = test.Review.apply(define_sentiment, test_all_abs=True)

In [32]:
print("This is the result without the word 'cyberpunk' included in the corpus:")
print(f"This is our accuracy for (balanced) synthetic data with all the formula: {(sum(test['Recommended or Not Recommended'] == test['score_all'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (balanced) synthetic data with only class probability, frequency, and absolute value of TF: {(sum(test['Recommended or Not Recommended'] == test['score_tf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (balanced) synthetic data with only class probability, frequency, and absolute value of IDF: {(sum(test['Recommended or Not Recommended'] == test['score_idf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (balanced) synthetic data with only class probability and frequency: {(sum(test['Recommended or Not Recommended'] == test['score_freq'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (balanced) synthetic data with all the formula, but with the absolute value of the score: {(sum(test['Recommended or Not Recommended'] == test['score_mod_all'])/test.shape[0])*100:.2f}%")

This is the result without the word 'cyberpunk' included in the corpus:
This is our accuracy for (balanced) synthetic data with all the formula: 51.02%
This is our accuracy for (balanced) synthetic data with only class probability, frequency, and absolute value of TF: 87.73%
This is our accuracy for (balanced) synthetic data with only class probability, frequency, and absolute value of IDF: 87.24%
This is our accuracy for (balanced) synthetic data with only class probability and frequency: 85.17%
This is our accuracy for (balanced) synthetic data with all the formula, but with the absolute value of the score: 89.96%


In [33]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_all'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_all']))


              precision    recall  f1-score   support

       False       0.50      0.62      0.55      1699
        True       0.53      0.41      0.46      1766

    accuracy                           0.51      3465
   macro avg       0.51      0.51      0.51      3465
weighted avg       0.51      0.51      0.51      3465



In [34]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_tf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_tf']))


              precision    recall  f1-score   support

       False       0.81      0.99      0.89      1699
        True       0.99      0.77      0.86      1766

    accuracy                           0.88      3465
   macro avg       0.90      0.88      0.88      3465
weighted avg       0.90      0.88      0.88      3465



In [35]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_idf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_idf']))


              precision    recall  f1-score   support

       False       0.80      0.99      0.88      1699
        True       0.98      0.76      0.86      1766

    accuracy                           0.87      3465
   macro avg       0.89      0.87      0.87      3465
weighted avg       0.89      0.87      0.87      3465



In [36]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_freq'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_freq']))


              precision    recall  f1-score   support

       False       0.77      0.99      0.87      1699
        True       0.99      0.72      0.83      1766

    accuracy                           0.85      3465
   macro avg       0.88      0.85      0.85      3465
weighted avg       0.88      0.85      0.85      3465



Bad pipe message: %s [b':O\xfd\x90\xd7R=\xfa\x0f\x80g\x04\xdd\x8c\xd66w\xb9 \x9a\xb8%))7]\xcc\x90&\xfd%\x7f\xae\xfa\x1fT\xd9\xa8\x0e\xc7\xd9\xf11"\x83M\x83\xe3\xe0f!\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 :\xb9|.\x063\'I}\no']
Bad pipe message: %s [b"\xb1\xaa\xbe)\xc9'*\xa3f\xba)4\xf912\xe0\x96\xda \xcd\xc19\x02\xde\xb9\x80\x08\xda\x11\xd5\xce\xed"]
Bad pipe message: %s [b'\xd4\x0cNZ\xc8?\xf1j5\xc0\xc7\xf3\x9d', b"v\xf71\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\